In [2]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import os

In [3]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores",8).getOrCreate()

In [4]:
path='D:\\Project_Python_(DA-DE)\\DE_Project\\BigData\\Class_5\\OLAP_OUTPUT\\Data\\'

In [5]:
list_path=os.listdir(path)


In [ ]:
df=spark.read.json(path+list_path[0])
for i in list_path[1:]:
    save_path=i
    read_path=spark.read.json(path+i)
    df=df.union(read_path)
    print('Successfully Processing {}'.format(i))
print('----Done----')

In [ ]:
df.show()

In [ ]:
processing=df.select("_source.*")

In [ ]:
processing.select('AppName').distinct().collect()

In [ ]:
processing=processing.withColumn('Type',
                      when ((col('AppName')=='CHANNEL') | (col('AppName')=='KPLUS')| (col('AppName')=='KPlus'),"Truyen Hinh")
                      .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyen")
        .when((col("AppName") == 'RELAX'), "Giai Tri")
        .when((col("AppName") == 'CHILD'), "Thieu Nhi")
        .when((col("AppName") == 'SPORT'), "The Thao")
        .otherwise("Error")
)


In [ ]:
processing.show()

In [ ]:
processing=processing.select('Contract','Type','TotalDuration')

In [ ]:
processing=processing.filter(processing.Type != 'Error')

In [ ]:
# ETL

processing=processing.groupBy('Contract','Type').sum('TotalDuration')

In [ ]:
processing.show()

In [ ]:
processing=processing.withColumnRenamed('sum(TotalDuration)','TotalDuration')

In [ ]:
processing=processing.groupBy('Contract').pivot('Type').sum('TotalDuration')

In [ ]:
processing=processing.na.fill(0)

In [ ]:
processing.show()

# Output_Processing_continue

## Most_Watch

In [ ]:
processing=processing.withColumn("MostWatch",greatest(col("Giai Tri"),col("Phim Truyen"),col("The Thao"),col("Thieu Nhi"),col("Truyen Hinh"),col("Giai Tri")))

In [ ]:
processing.drop('MostWacth')

In [ ]:
processing=processing.withColumn('MostWatch',
                    when(col('MostWatch')==col('Truyen Hinh'),'Truyen Hinh')
                    .when(col('MostWatch')==col('Thieu Nhi'),'Thieu Nhi')
                    .when(col('MostWatch')==col('The Thao'),'The Thao')
                    .when(col('MostWatch')==col('Phim Truyen'),'Phim Truyen')
                    .when(col('MostWatch')==col('Giai Tri'),'Giai Tri')
)

## Customer_Taste

In [ ]:
processing=processing.withColumn("Taste",concat_ws("-",
                                    when(col("Giai Tri").isNotNull(),lit("Giai Tri"))
                                    ,when(col("Phim Truyen").isNotNull(),lit("Phim Truyen"))
                                    ,when(col("The Thao").isNotNull(),lit("The Thao"))
                                    ,when(col("Thieu Nhi").isNotNull(),lit("Thieu Nhi"))
                                    ,when(col("Truyen Hinh").isNotNull(),lit("Truyen Hinh"))))

## ActiveNess

In [6]:
df=spark.read.json(path+list_path[0])
df=df.withColumn("Path",
              lit("{}".format(list_path[0])))
df=df.withColumn("Date", split("Path", ".json").getItem(0))
for i in list_path[1:]:
    save_path=i
    read_path=spark.read.json(path+save_path)
    read_path=read_path.withColumn("Path", 
                                   lit("{}".format([save_path])))
    read_path=read_path.withColumn("Date", split('Path', ".json").getItem(0))
    df=df.union(read_path)
    print('Successfully Processing {}'.format(i))
df=df.drop("Path")  

print('------------Done------------------')


Successfully Processing 20220402.json
Successfully Processing 20220403.json
Successfully Processing 20220404.json
Successfully Processing 20220405.json
Successfully Processing 20220406.json
Successfully Processing 20220407.json
Successfully Processing 20220408.json
Successfully Processing 20220409.json
Successfully Processing 20220410.json
Successfully Processing 20220411.json
Successfully Processing 20220412.json
Successfully Processing 20220413.json
Successfully Processing 20220414.json
Successfully Processing 20220415.json
Successfully Processing 20220416.json
Successfully Processing 20220417.json
Successfully Processing 20220418.json
Successfully Processing 20220419.json
Successfully Processing 20220420.json
Successfully Processing 20220421.json
Successfully Processing 20220422.json
Successfully Processing 20220423.json
Successfully Processing 20220424.json
Successfully Processing 20220425.json
Successfully Processing 20220426.json
Successfully Processing 20220427.json
Successfully

In [7]:
from pyspark.sql.window import Window

In [17]:
activeness=df.select('_source.*','Date')

In [18]:
window=Window.partitionBy("Contract")

In [19]:
activeness=activeness.withColumn("Active",size(collect_set("Date").over(window)))

In [20]:
activeness

DataFrame[AppName: string, Contract: string, Mac: string, TotalDuration: bigint, Date: string, Active: int]

In [21]:
activeness=activeness.withColumn("Activeness",
                      when(col('Active')>=15,"High Active")
                      .otherwise("Low Active")
)

In [37]:
distinct_activeness = activeness.dropDuplicates(["Contract", "Activeness"])


In [39]:
distinct_activeness.show()

+-------+--------------+------------+-------------+----------+------+-----------+
|AppName|      Contract|         Mac|TotalDuration|      Date|Active| Activeness|
+-------+--------------+------------+-------------+----------+------+-----------+
|CHANNEL|113.182.209.48|B046FCA97B6A|           20|  20220401|     1| Low Active|
|CHANNEL|14.182.110.125|B84DEED3BD63|          404|['20220410|     1| Low Active|
|CHANNEL|     AGAAA0338|10394E3D6E3C|         8895|  20220401|    30|High Active|
|CHANNEL|     AGAAA0342|10394E3D7A67|           44|  20220401|    12| Low Active|
|CHANNEL|     AGAAA0391|10394E3D6E9F|        29102|['20220402|    11| Low Active|
|CHANNEL|     AGAAA0613|10394E3D6DBF|           83|  20220401|    24|High Active|
|CHANNEL|     AGAAA0638|10394E3D6F2C|          382|  20220401|    30|High Active|
|CHANNEL|     AGAAA0692|10394E3D8C74|        14551|['20220406|     8| Low Active|
|CHANNEL|     AGAAA0723|10394E3D84FF|           27|  20220401|    19|High Active|
|CHANNEL|     AG